In [12]:
import random
import numpy as np
import queueing_tool as qt
import pandas as pd

Define service function

In [13]:
def service_f(t: float):
    """Define deterministic service time.\n
    Take the arrival time as t and add a constant.

    Args:
        t (float): current time
    """
    return t + 0.0000012

def identity(t: float):
    """Handler function for returning the current time

    Args:
        t (float): current time

    Returns:
        (float): current time
    """
    return t

Define arrival functions

In [14]:
def arr_slow(t: float):
    """Return arrival time with slow rate

    Args:
        t (float): current time

    Returns:
        (float): arrival time
    """
    return t + random.expovariate(lambd=125)
    # return t + random.expovariate(lambd=1.0/33333.3333)

def arr_fast(t: float):
    """Return arrival time with fast rate

    Args:
        t (float): current time

    Returns:
        (float): arrival time
    """
    return t + random.expovariate(lambd=500)
    # return t + random.expovariate(lambd=1.0/633333.667)

Define the two agents:
- slow agents that go through the whole system
- fast agents that overload the system and leave

In [15]:
class FastAgent(qt.Agent):
    def __init__(self, agent_id=(0, 0)):
        super().__init__(agent_id)
        self.agent_id = (agent_id[0], agent_id[1], 'fast')
        
class SlowAgent(qt.Agent):
    def __init__(self, agent_id=(0, 0)):
        super().__init__(agent_id)
        self.agent_id = (agent_id[0], agent_id[1], 'slow')

System definition

```mermaid
---
title: One-node M/D/1 system
---
  graph LR;
  A("Entry point")
  B("Leaving point")
  λ0("slow λ")
  λ1("fast λ")

      λ0 --> A
      λ1 --> node
      A --> node --> B
```

Network definition

In [16]:
# Prepare the one-node network
q_classes = { 1: qt.QueueServer, 2: qt.QueueServer, 2: qt.QueueServer }

adja_list = {
    0: [2],
    1: [2],
    2: [3]
}
edge_list = {
    0: {2: 1},
    1: {2: 2},
    2: {3: 3}
}
g = qt.adjacency2graph(
    adjacency=adja_list,
    edge_type=edge_list
)

q_args = {
    1: {
        'arrival_f': arr_slow,
        'service_f': identity,
        'AgentFactory': SlowAgent,
    },
    2: {
        'arrival_f': arr_fast,
        'service_f': identity,
        'AgentFactory': FastAgent,
    },
    3: {
        'service_f': service_f,
    },
}

qn = qt.QueueNetwork(
    g= g, q_classes=q_classes, q_args=q_args
)

Simulate

In [17]:
qn.initialize(edge_type=[0, 1, 2])
qn.start_collecting_data()
qn.simulate(n=200)
dat = qn.get_agent_data(return_header=True)

Store values

In [18]:
slow_l = []
fast_l = []
for k, v in dat[0].items():
    if k[2] == 'slow':
        slow_l.append({k: v})
    if k[2] == 'fast':
        fast_l.append({k: v})

In [57]:
# tranform the requests list of dictionaries into dictionary
slow_d = {k[1]: v for req in slow_l for k, v in req.items() if len(v) > 2}
fast_d = {k[1]: v for req in fast_l for k, v in req.items() if len(v) > 2}

First slow request

In [60]:
print([*list(slow_d.items())[-1][1][0]])

[0.09529162971863198, 0.09529162971863198, 0.09529162971863198, 0.0, 1.0, 0.0]


Last slow request

In [43]:
[*list(slow_d.items())[-1][1][2]]

[0.09529282971863198, 0.0, 0.0, 0.0, 0.0, 3.0]

Calculate:
- last departure - first arrival

In [56]:
format(
    float(
        [*list(slow_d.items())[-1][1][2]][0] - [*list(slow_d.items())[-1][1][0]][0],
    ), '.9f'
)

'0.000001200'

Check the same for fast

In [58]:
format(
    float(
        [*list(fast_d.items())[-1][1][2]][0] - [*list(fast_d.items())[-1][1][0]][0],
    ), '.9f'
)

'0.000001200'